### 1.0.0
<p>2019-10-21 初版完成，輸入拍賣日期範圍，即可抓取交易資料，並自動將資料整理為符合模板的格式</p>

### 1.0.1 
<p>2020-1-16 將參數中 "QUERY_BTN1	: 查 詢" 刪除，才可控制頁數以及每一頁顯示的資料數</p>

### 1.0.2 
<p>2020-7-8 新增參數 "Q_OUTSIDE_POINT$0~8"，將其設為"on"才能正確爬取資料</p>

In [5]:
# 載入套件
import requests
import numpy as np
import time
import datetime
import pandas as pd
from io import BytesIO, StringIO
from bs4 import BeautifulSoup as bs

# -------------------------------登入行將-------------------------------- #
# 使用session並網頁登入
session_requests = requests.session()
login_url = "https://www.sinjang.com.tw/Portal/"
result = session_requests.get(login_url)
soup = bs(result.text)
soup

# 抓取csrf_token及其他request所需資料
viewstate = soup.find(id="__VIEWSTATE")['value']
viewstategenerator = soup.find(id="__VIEWSTATEGENERATOR")['value']
validation = soup.find(id="__EVENTVALIDATION")['value']

# 輸入post的param及header
payload = {
    'ctl00$ScriptManager1': 'ctl00$BodyContent$AjaxPanel|ctl00$BodyContent$LOGIN_BTN',
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': viewstate,
    '__VIEWSTATEGENERATOR': viewstategenerator,
    '__VIEWSTATEENCRYPTED': '',
    '__EVENTVALIDATION': validation,
    'ctl00$BodyContent$H_MONTH': '10',
    'ctl00$BodyContent$H_YEAR': '2019',
    'ctl00$BodyContent$monthflag': '',
    'ctl00$BodyContent$H_HAPPY_URL': 'http://fun.sinjang.com.tw/',
    'ctl00$BodyContent$hidCOUNTDOWN': '',
    'ctl00$BodyContent$hidALERTTIME': '',
    'ctl00$BodyContent$hidTIMEFLAG': '',
    'ctl00$BodyContent$timer_idx': '',
    'ctl00$BodyContent$ACCOUNT': '帳號',
    'ctl00$BodyContent$PASSWORD': '密碼',
    '__ASYNCPOST': 'true',
    'ctl00$BodyContent$LOGIN_BTN.x': '36',
    'ctl00$BodyContent$LOGIN_BTN.y': '61'
}

header = {
    'Host': 'www.sinjang.com.tw',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.sinjang.com.tw/Portal/AUC2101_.aspx',
    'X-MicrosoftAjax': 'Delta=true',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
    'Content-Length': '14209',
    'Connection': 'keep-alive',
}

# 完成登入，並將登入後頁面存入result中
result = session_requests.post(
    login_url,
    data=payload,
    headers=header
)

# 檢視是否登入成功
bs(result.text)


# -----------------------------完成登入後爬取資料-------------------------- #
# 登入成功後使用同一個session去爬取我們需要的頁面
crw_url = 'https://www.sinjang.com.tw/Portal/BA0102_01.aspx'
crw_result = session_requests.get(crw_url)

# 抓取csrf_token及其他request所需資料
crw_soup = bs(crw_result.text)
crw_viewstate = crw_soup.find(id="__VIEWSTATE")['value']
crw_viewstategenerator = crw_soup.find(id="__VIEWSTATEGENERATOR")['value']
crw_pageid = crw_soup.find(id="PAGE_ID")['value']
crw_validation = crw_soup.find(id="__EVENTVALIDATION")['value']

# 輸入爬取參數 : 交易時間
BID_DATE_S = input('Start_date : yyyy/m/d \n')
BID_DATE_S_F = datetime.datetime.strptime(BID_DATE_S, '%Y/%m/%d').strftime('%Y%m%d')
BID_DATE_E = input('End_date : yyyy/m/d \n')
BID_DATE_E_F = datetime.datetime.strptime(BID_DATE_E, '%Y/%m/%d').strftime('%Y%m%d')
# BRANDS = ['AUDI', 'PORSCHE', 'SKODA', 'VW']
BRANDS = ['05', '56', '64', '73']
maindf = pd.DataFrame()

for BRAND in BRANDS:
    # 輸入post的param及header
    crw_param = {
        "__ASYNCPOST":"true",
        "__EVENTARGUMENT":"",
        "__EVENTTARGET":"ReQuery",
        "__EVENTVALIDATION":crw_validation,
        "__LASTFOCUS":"",
        "__VIEWSTATE": crw_viewstate,
        "__VIEWSTATEENCRYPTED":"",
        "__VIEWSTATEGENERATOR":crw_viewstategenerator,
        "ActivePageControl":"PC2",
        "ColumnFilter":"",
        "M_PKNO":"",
        "Mode":"ADD",
        "PAGE_ID":crw_pageid,
        "PASSWORD":"",
        "PASSWORD_C":"",
        "PC$PageNo":"1",
        "PC$PageSize":"99999",
        "PC2$PageNo":"1",
        "PC2$PageSize":"99999",
        "Q_ALL_OUTSIDE_POINT":"on",
        "Q_BID_DATE_E":BID_DATE_E,
        "Q_BID_DATE_S":BID_DATE_S,
        "Q_BRAND_ID":BRAND,
        "Q_CAR_AGE_E":"",
        "Q_CAR_AGE_S":"",
        "Q_CAR_DOOR":"",
        "Q_GEAR_TYPE":"",
        "Q_MODEL_ID":"",
        "Q_OIL_TYPE":"",
        "Q_OUTSIDE_POINT$0":"on",
        "Q_OUTSIDE_POINT$1":"on",
        "Q_OUTSIDE_POINT$2":"on",
        "Q_OUTSIDE_POINT$3":"on",
        "Q_OUTSIDE_POINT$4":"on",
        "Q_OUTSIDE_POINT$5":"on",
        "Q_OUTSIDE_POINT$6":"on",
        "Q_OUTSIDE_POINT$7":"on",
        "Q_OUTSIDE_POINT$8":"on",
        "Q_SPEEDOMETER":"",
        "Q_TOLERANCE_E":"",
        "Q_TOLERANCE_S":"",
        "Q_WD":"",
#         "QUERY_BTN1":"查 詢",
        "ROWSTAMP":"",
        "ScriptManager1":"AjaxPanel|ReQuery",
        "TAB":"1",
        "TYPE":"",
    }

    crw_header = {
        'Host': "www.sinjang.com.tw"
        ,'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0"
        ,'Accept': "*/*"
        ,'Accept-Language': "en-US,en;q=0.5"
        ,'Accept-Encoding': "gzip, deflate, br"
        ,'Referer': "https://www.sinjang.com.tw/Portal/BA0102_01.aspx"
        ,'X-MicrosoftAjax': "Delta=true"
        ,'Cache-Control': "no-cache"
        ,'Content-Type': "application/x-www-form-urlencoded; charset=utf-8"
#         ,'Content-Length': "27013"
#         ,'Cookie': "_ga=GA1.3.1490414022.1571654339; ASP.NET_SessionId=cdlkyyfmnqui1uq1pds5bkrs"
        ,'Connection': "keep-alive"
    }

    # 送出請求
    crw_result = session_requests.post(
        crw_url,
        data=crw_param,
        headers=crw_header
    )
    
   
    
    # 使用pandas讀取html
    table = pd.read_html(StringIO(crw_result.text))
    
    # 找出 table 中最欄數最多的DF，才是我們需要的DF
    max_col_df = 0
    max_df_num = 0
    for num_df, df in enumerate(table):
        if len(df.columns) > max_col_df:
            max_col_df = len(df.columns)
            max_df_num = num_df
    
    maindf = maindf.append(table[9])
    time.sleep(3)
    
# -----------------------------資料整理-------------------------- #   
column = ['拍賣日期', '頻道', '拍賣編號', '查看資料', '廠牌', '車型', '監理型式', '式樣', '排檔', '出廠年月', '排氣量', '顏色',
          '傳動方式', '車門', '車體評價', '內裝評價', '里程', '里程保證', '成交價', '牌照稅', '燃料費', '違規',
          '成交價(調整)', '排氣量(調整)']
maindf['頻道'] = None
maindf['拍賣編號'] = None
maindf['拍賣日期'] = pd.to_datetime(maindf.拍賣日期)
maindf['成交價(調整)'] = maindf.成交價 + maindf.牌照稅 + maindf.燃料費 + maindf.違規
maindf['排氣量(調整)'] = np.ceil(maindf.排氣量 / 100) / 10
maindf['廠牌'] = maindf.廠牌車型.str.split(' ').apply(lambda x: x[0])
maindf['車型'] = maindf.廠牌車型.str.split(' ').apply(lambda x: x[2])
maindf.drop('廠牌車型', inplace=True, axis=1)
maindf[column].to_excel('C:\行將交易資料\行將_' + BID_DATE_S_F + '-' + BID_DATE_E_F + '.xlsx', index=False)
print('----------------------------------Finished----------------------------------------')    

Start_date : yyyy/m/d 
2020/4/1
End_date : yyyy/m/d 
2020/6/30
----------------------------------Finished----------------------------------------
